In [6]:
import Pkg
Pkg.add("FilePathsBase")
using FilePathsBase
rel_path_to_GenX = relpath(joinpath(homedir(), "GenX"))
Pkg.activate("../../../../GenX")
Pkg.status()

   Resolving package versions...
  No Changes to `~/GenX/Project.toml`
  No Changes to `~/GenX/Manifest.toml`
Precompiling project...
  ✓ GenX
  1 dependency successfully precompiled in 11 seconds. 288 already precompiled.
  Activating project at `~/GenX`


Project GenX v0.4.0
Status `~/GenX/Project.toml`
⌃ [336ed68f] CSV v0.10.14
⌃ [159f3aea] Cairo v1.0.5
  [aaaa29a8] Clustering v0.15.7
  [861a8166] Combinatorics v1.0.2
⌃ [a93c6f00] DataFrames v1.6.1
  [864edb3b] DataStructures v0.18.20
  [7db386b5] DependencyWalker v0.1.2
⌃ [b4f34e82] Distances v0.10.11
  [48062228] FilePathsBase v0.9.22
  [4c0ca9eb] Gtk v1.3.0
⌃ [87dc4568] HiGHS v1.9.0
⌃ [7073ff75] IJulia v1.24.2
⌃ [4076af6c] JuMP v1.22.0
⌃ [b8f27783] MathOptInterface v1.29.0
⌃ [f0f68f2c] PlotlyJS v0.18.13
⌃ [91a5bcdd] Plots v1.40.4
⌃ [731186ca] RecursiveArrayTools v3.18.1
  [3646fa90] ScikitLearn v0.7.0
⌅ [2913bbd2] StatsBase v0.33.21
  [112f6efa] VegaLite v3.3.0
⌃ [ddb6d928] YAML v0.4.9
⌃ [83423d85] Cairo_jll v1.18.0+2
⌃ [36c8627f] Pango_jll v1.52.2+0
  [30392449] Pixman_jll v0.43.4+0
  [ade2ca70] Dates
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
  [10745b16] Statistics v1.10.0
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those

In [7]:
using JuMP, HiGHS
using Plots
using DataFrames, CSV#, PrettyTables

In [8]:
datadir = joinpath("OPF_data") 
gen = CSV.read(joinpath(datadir,"gen.csv"), DataFrame);
gencost = CSV.read(joinpath(datadir,"gencost.csv"), DataFrame);
branch = CSV.read(joinpath(datadir,"branch.csv"), DataFrame);
bus = CSV.read(joinpath(datadir,"bus.csv"), DataFrame);
for f in [gen, gencost, branch, bus]
    rename!(f, lowercase.(names(f)))
end

gen.id = 1:nrow(gen);
gencost.id = 1:nrow(gencost);
branch.id = 1:nrow(branch);
# Reverse direction with same parameters
branch2 = copy(branch)
branch2.f = branch2.fbus
branch2.fbus = branch2.tbus
branch2.tbus = branch2.f
branch2 = branch2[:,names(branch)]
append!(branch,branch2)

# Calculate the susceptance of each line, on the assumption that
# reactance is >> resistance, such that we can approximate 
# resistance as = 0 and treat susceptance as the simple  
# reciprocal of reactance (x). 
# See https://en.wikipedia.org/wiki/Susceptance#Relationship_to_reactance
branch.sus = 1 ./ branch.x    

6-element Vector{Float64}:
 35.587188612099645
 35.587188612099645
 35.587188612099645
 35.587188612099645
 35.587188612099645
 35.587188612099645

In [9]:
# Bus set
bus

Row,bus_i,type,pd,qd,gs,bs,area,vm,va,basekv,zone,vmax,vmin
,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64
1,1,2,0,0.0,0,0,1,1,0,230,1,1.1,0.9
2,2,2,0,0.0,0,0,1,1,0,230,1,1.1,0.9
3,3,1,600,98.61,0,0,1,1,0,230,1,1.1,0.9


In [10]:
# Generator set
gen

Row,bus,pg,qg,qmax,qmin,vg,mbase,status,pmax,pmin,pc1,pc2,qc1min,qc1max,qc2min,qc2max,ramp_agc,ramp_10,ramp_30,ramp_q,apf,id
,Int64,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,40,0,30.0,-30.0,1,100,1,1000,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,170,0,127.5,-127.5,1,100,1,1000,0,0,0,0,0,0,0,0,0,0,0,0,2


In [11]:
# Generator cost set
gencost

Row,model,startup,shutdown,n,x1,y1,id
,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,2,0,0,2,50,0,1
2,2,0,0,2,100,0,2


In [29]:
# Transmission lines, with a row for each direction
branch

Row,fbus,tbus,r,x,b,ratea,rateb,ratec,ratio,angle,status,angmin,angmax,id,sus
,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,1,3,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,1,35.5872
2,1,2,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,2,35.5872
3,2,3,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,3,35.5872
4,3,1,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,1,35.5872
5,2,1,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,2,35.5872
6,3,2,0.00281,0.0281,0.00712,500,500,500,0,0,1,-360,360,3,35.5872


In [32]:
i = 1
j = 3
#branch[branch.tbus .== i, :fbus] # returns all fbus values from rows in branch where tbus == 1
#branch[branch.tbus .== i, 3] # returns all r values (3rd index) from rows in branch where tbus == 1
#branch[branch.tbus .== i, :] # returns all values from rows in branch where tbus == 1
# I want sus values such that tbus == i and fbus == j
branch[(branch.fbus .== i) .& (branch.tbus .== j), :sus]

1-element Vector{Float64}:
 35.587188612099645

In [53]:
#=
Function to solve transport flow problem 
Inputs:
    gen -- dataframe with generator info
    branch -- dataframe with transmission lines info
    gencost -- dataframe with generator info
    bus -- dataframe with bus types and loads
=#
function dc_opf(gen, branch, gencost, bus)
    Transport = Model(HiGHS.Optimizer)

    BaseMVA = bus[:,:basekv][1]
    
    # Sets
    Generators = gen.id
    Nodes = bus.bus_i

    # Decision variables
    @variables(Transport, begin
        GEN[Generators] >= 0
        FLOW[Nodes,Nodes]
        Theta[Nodes]
    end)

    # Objective function
    # Minimize the sum of all generator costs
    @objective(Transport, 
        Min,
        sum(gencost[g,:x1] * GEN[g] for g in Generators)
    )

    # Supply / demand balance constraints
    @constraint(Transport, cBalance[i in Nodes],
        sum(GEN[g] for g in gen[gen.bus .== i, :id]) - bus[bus.bus_i .== i, :pd][1] .== 
        sum(FLOW[i,j] for j in branch[branch.tbus .== i, :fbus]))

    # Max generation constraints
    @constraint(Transport, cMaxGen[g in Generators], GEN[g] <= gen[g,:pmax])

    # Flow constraints
    for l in 1:nrow(branch)
        @constraint(Transport, FLOW[branch[l,:fbus][1],branch[l,:tbus][1]] <= branch[l, :ratea])
    end

    ##Anti-symmetric flow constraints
    #@constraint(Transport, cFlowSymmetric[i in Nodes, j in Nodes], FLOW[i,j] == -FLOW[j,i])
    # Flow from phase angle difference
    @constraint(Transport, cFlowFromPhase[i in Nodes, j in Nodes], FLOW[i,j] 
        # == BaseMVA*branch[(branch.fbus .== i) .& (branch.tbus .== j), :sus] # add phase angle difference here
        == BaseMVA * (isempty(branch[(branch.fbus .== i) .& (branch.tbus .== j), :sus]) ? 0.0 : only(branch[(branch.fbus .== i) .& (branch.tbus .== j), :sus]))
        
    )
    #@constraint(Transport, cSlack, theta_slack==0)
    
    optimize!(Transport)

    # Dataframe of optimal decision variables
    generation = DataFrame(
        id = gen.id,
        node = gen.bus,
        gen = value.(GEN).data
    )
    flows = value.(FLOW).data

    return (
        generation = generation, 
        flows,
        cost = objective_value(Transport),
        status = termination_status(Transport)
    )
end

dc_opf (generic function with 1 method)

In [54]:
solution = dc_opf(gen, branch, gencost, bus)
solution.generation

Running HiGHS 1.7.0 (git hash: 50670fd4c): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [5e+01, 1e+02]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 1e+03]
Presolving model
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
ERROR:   No LP invertible representation for getDualRay


LoadError: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.

In [41]:
solution.flows

3×3 Matrix{Float64}:
   -0.0   100.0  500.0
 -100.0    -0.0  100.0
 -500.0  -100.0   -0.0